In [1]:
import pandas as pd
import numpy as np

WIND_CAPACITY = 200 #MWh

spot_data = pd.read_csv('Data/DK1_DA_prices_2023.csv')[['HourDK', 'SpotPriceEUR']] #load day-ahead price data from Energinet file (March 2024)
spot_data['HourDK'] = pd.to_datetime(spot_data['HourDK']) #convert 'HourUTC' to datetime format
spot_data = spot_data.set_index('HourDK') #set 'HourDK' as index
spot_data['Day'] = spot_data.index.day #extract day from 'HourUTC'

In [2]:
production_data = pd.read_csv('Data/Energinet_Production_Data_2023.csv')[['Minutes5UTC', 'Minutes5DK', 'OffshoreWindPower', 'OnshoreWindPower']] #load production data from Energinet file (2023)
production_data['Wind'] = production_data['OffshoreWindPower'] + production_data['OnshoreWindPower'] #sum offshore and onshore wind power

production_data = production_data.rename(columns={'Minutes5DK': 'Time_DK'}) 
production_data['Time_DK'] = pd.to_datetime(production_data['Time_DK'])
production_data = production_data.set_index('Time_DK')

production_data = production_data['Wind'].to_frame()
production_data = production_data.resample('1h').mean() #resample data to hourly frequency (mean)
production_data['Wind'] = (production_data['Wind'] / production_data['Wind'].max()) * WIND_CAPACITY #scale wind data to the wind farm capacity
production_data['Day'] = production_data.index.day
production_data

,Wind,Day
Time_DK,,
2023-01-01 00:00:00,28.556556,1
2023-01-01 01:00:00,45.864962,1
2023-01-01 02:00:00,46.663636,1
2023-01-01 03:00:00,46.936729,1
2023-01-01 04:00:00,45.669398,1
...,...,...
2023-12-31 19:00:00,136.875626,31
2023-12-31 20:00:00,124.043065,31
2023-12-31 21:00:00,115.103740,31


In [3]:
spot_data

,SpotPriceEUR,Day
HourDK,,
2023-01-01 00:00:00,2.010000,1
2023-01-01 01:00:00,1.380000,1
2023-01-01 02:00:00,0.090000,1
2023-01-01 03:00:00,0.080000,1
2023-01-01 04:00:00,0.050000,1
...,...,...
2023-12-31 19:00:00,39.639999,31
2023-12-31 20:00:00,34.889999,31
2023-12-31 21:00:00,29.600000,31


In [4]:
start_time = '2023-12-01 00:00:00'
end_time = '2023-12-31 23:00:00'

wind_data = production_data.loc[(production_data.index >= start_time) & (production_data.index <= end_time)].copy(deep=True)
spot = spot_data.loc[(spot_data.index >= start_time) & (spot_data.index <= end_time)].copy(deep=True)

wind_scenarios = {}
price_scenarios = {}

for day in range(1, 21): #20 scenarios for wind and price data
    #using zero-indexing (from "day" to a scenario index)
    wind_scenarios[day - 1] = list(wind_data.loc[wind_data.Day == day, 'Wind'].values)
    price_scenarios[day - 1] = list(spot.loc[spot_data.Day == day, 'SpotPriceEUR'].values)

    print(day, ":" ,len(wind_scenarios[day - 1]), len(price_scenarios[day - 1]))

1 : 24 24
2 : 24 24
3 : 24 24
4 : 24 24
5 : 24 24
6 : 24 24
7 : 24 24
8 : 24 24
9 : 24 24
10 : 24 24
11 : 24 24
12 : 24 24
13 : 24 24
14 : 24 24
15 : 24 24
16 : 24 24
17 : 24 24
18 : 24 24
19 : 24 24
20 : 24 24


In [5]:
system_state_scenarios = {}

for k in range(5):
    np.random.seed(k) #set seed of random generator
    system_state_scenarios[k] = np.random.binomial(1, 0.5, 24).astype(int).tolist() #generate 24 random binary numbers from binomial distribution with p=0.5 (1 draw)
    print(k, ":", system_state_scenarios[k])

0 : [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1]
1 : [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1]
2 : [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
3 : [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0]
4 : [1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1]


In [6]:
scenario_data = {
    'Wind Power [MW]': wind_scenarios,
    'Spot Price [EUR/MWh]': price_scenarios,
    'System Balance State': system_state_scenarios
}

#Write to .json
import json
with open('Data/scenario_data.json', 'w') as f:
    json.dump(scenario_data, f)

Generate all possible combinations of wind, price and system states

In [7]:
import itertools

ranges = [range(len(wind_scenarios)), range(len(price_scenarios)), range(len(system_state_scenarios))]

all_scenario_combinations = list(itertools.product(*ranges)) # "*" unpacks the list of ranges into the function argument

print('Number of combinations:',len(all_scenario_combinations))

#all_scenario_combinations #display?

Number of combinations: 2000


Export all scenarios

In [ ]:
all_scenarios = {}

#Add the list of scenario indices for wind power, spot price, and system balance state to document the in-sample scenarios
all_scenarios['Sample Scenarios'] = {'Wind Scenario Index':{}, 'Price Scenario Index':{}, 'System State Scenario Index':{}} #initialize
all_scenarios['Sample Scenarios']['Wind Scenario Index'] = [combination[0] for combination in all_scenario_combinations] #store the indices of the wind power scenarios
all_scenarios['Sample Scenarios']['Price Scenario Index'] = [combination[1] for combination in all_scenario_combinations] #store the indices of the spot price scenarios
all_scenarios['Sample Scenarios']['System State Scenario Index'] = [combination[2] for combination in all_scenario_combinations] #store the indices of the system balance state scenarios


k = 0
for combination in all_scenario_combinations:
    w, p, s = combination

    all_scenarios[k] = {
        'Wind Power [MW]': wind_scenarios[w],
        'Spot Price [EUR/MWh]': price_scenarios[p],
        'System Balance State': system_state_scenarios[s]
    }

    k += 1

with open('Data/ALL_scenarios.json', 'w') as f:
    json.dump(all_scenarios, f)

# For testing: Generate a larger number of system state scenarios

In [24]:
N = 20

system_state_scenarios = {}

for k in range(N):
    np.random.seed(k+1) #set seed of random generator
    system_state_scenarios[k] = np.random.binomial(1, 0.5, 24).astype(int).tolist() #generate 24 random binary numbers from binomial distribution with p=0.5 (1 draw)
    print(k, ":", system_state_scenarios[k])

print(np.mean([system_state_scenarios[k]for k in range(N)], axis=0))

0 : [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1]
1 : [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
2 : [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0]
3 : [1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1]
4 : [0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
5 : [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1]
6 : [0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]
7 : [1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
8 : [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
9 : [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1]
10 : [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
11 : [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
12 : [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,

In [28]:
import itertools, json

ranges = [range(len(wind_scenarios)), range(len(price_scenarios)), range(len(system_state_scenarios))]

all_scenario_combinations = list(itertools.product(*ranges)) # "*" unpacks the list of ranges into the function argument

print('Number of combinations:',len(all_scenario_combinations))

#all_scenario_combinations #display?

Number of combinations: 8000


In [29]:
all_scenarios = {}

#Add the list of scenario indices for wind power, spot price, and system balance state to document the in-sample scenarios
all_scenarios['Sample Scenarios'] = {'Wind Scenario Index':{}, 'Price Scenario Index':{}, 'System State Scenario Index':{}} #initialize
all_scenarios['Sample Scenarios']['Wind Scenario Index'] = [combination[0] for combination in all_scenario_combinations] #store the indices of the wind power scenarios
all_scenarios['Sample Scenarios']['Price Scenario Index'] = [combination[1] for combination in all_scenario_combinations] #store the indices of the spot price scenarios
all_scenarios['Sample Scenarios']['System State Scenario Index'] = [combination[2] for combination in all_scenario_combinations] #store the indices of the system balance state scenarios


k = 0
for combination in all_scenario_combinations:
    w, p, s = combination

    all_scenarios[k] = {
        'Wind Power [MW]': wind_scenarios[w],
        'Spot Price [EUR/MWh]': price_scenarios[p],
        'System Balance State': system_state_scenarios[s]
    }

    k += 1

with open('Data/ALL_scenarios_TESTING.json', 'w') as f:
    json.dump(all_scenarios, f)